In [ ]:
!pip install adabelief-pytorch==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
! pip install transformers
import os
from imblearn.over_sampling import RandomOverSampler
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
#from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Package 설치 & 데이터 받기
!pip install tensorflow==2.2.0
!pip install PyYAML==5.4.1

try:
    import transformers, emoji, soynlp, pytorch_lightning
except:
    !pip install -U -q transformers emoji soynlp pytorch-lightning

import os

if not os.path.exists('./nsmc'):
   !git clone https://github.com/e9t/nsmc

# !git clone https://github.com/Kim-se-yeon/o2o
# !head o2o/naver_shopping_train.txt

In [ ]:
# 패키지 import & 기본 Args 설정
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import re
import emoji
from soynlp.normalizer import repeat_normalize
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer

In [ ]:
#random seed 고정
tf.random.set_seed(4902)
np.random.seed(4902)
BATCH_SIZE = 64
NUM_EPOCHS = 10 #

L_RATE = 1e-5
MAX_LEN = 32
max_grad_norm=1
log_interval=200
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base', cache_dir='bert_ckpt', do_lower_case=False)
#https://github.com/monologg/KoELECTRA

In [ ]:
# 데이터셋 불러오기
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/아이펠톤/감성대화/data_processing/data_processing.xlsx') # df = pd.read_csv('/content/drive/Shareddrives/BOAZ_Adv/sentiment_label3.csv')

In [ ]:
df

In [ ]:
df.columns = ['content',	'emotion',	'label']

In [ ]:
df.emotion.value_counts()

In [ ]:
df.head(300)

In [ ]:
df.loc[(df['emotion'] == "기쁨"), 'emotion'] = 0  #기쁨 => 0
df.loc[(df['emotion'] == "긴장"), 'emotion'] = 1  #긴장 => 1
df.loc[(df['emotion'] == "평화"), 'emotion'] = 2  #평화 => 2
df.loc[(df['emotion'] == "슬픔"), 'emotion'] = 3  #슬픔 => 3
df.loc[(df['emotion'] == "분노"), 'emotion'] = 4  #분노 => 4
df.loc[(df['emotion'] == "중립"), 'emotion'] = 5  #중립 => 5

data_list = []
for q, label in zip(df['content'], df['emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(df, stratify=df['emotion'], test_size=0.25, random_state=0, shuffle = True)  #  stratify = y_label, // train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
dataset_test = dataset_test.reset_index()
dataset_test = dataset_test[[ 'content',	'emotion',	'label' ]]
dataset_test

In [ ]:
### train 에 대해서만 randomoversampling 진행
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler() ###
content_t = dataset_train.content.to_numpy().reshape(-1,1)
labels_t = dataset_train.emotion.to_numpy().reshape(-1,1)
labels_t=labels_t.astype('int')
X_over, y_over = oversample.fit_resample(content_t, labels_t)
dataset_train = pd.DataFrame({'content' : X_over.reshape(-1),'emotion' : y_over.reshape(-1)})

In [ ]:
labels_t

In [ ]:
dataset_test.emotion.value_counts()

In [ ]:
dataset_train.emotion.value_counts()

In [ ]:
# Koelectra 모델의 입력으로 들어갈 수 있게 형태 변환해주는 class
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
        self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", cache_dir='bert_ckpt', do_lower_case=False) #
    def __getitem__(self, index):
        # get the content from the dataframe
        content = self.df_data.loc[index, 'content'] # content = self.df_data[self.title].iloc[index] # 
        encoded_dict = tokenizer(
          text = content,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          padding='max_length',
          truncation=True,           # Pad & truncate all contents.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "emotion"]) # target = torch.tensor(self.df_data[self.topic_idx].iloc[index]) # # target = torch.tensor(self.df_data[self.topic_idx].iloc[index]) #   # 
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_data =TrainDataset(dataset_train) 
test_data = TrainDataset(dataset_test)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)

In [ ]:
!pip install adabelief-pytorch==0.2.0
from adabelief_pytorch import AdaBelief

model = AutoModelForSequenceClassification.from_pretrained('beomi/KcELECTRA-base',num_labels=6)


####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=229: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = False
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
from tqdm import tqdm
for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in tqdm(enumerate(train_dataloader)) : # enumerate(tqdm_notebook(train_dataloader))
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} train loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm(test_dataloader)) :
            input_id = input_id.long().to(device)
            token_type_id = token_type_id.long().to(device)
            attention_mask = attention_mask.long().to(device)
            label = label.to(device)
            outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
            loss = outputs[0]
            out = outputs[1]
            #loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            #optimizer.step()
            #scheduler.step()  # Update learning rate schedule
            
            test_acc += calc_accuracy(out, label)
            if batch_id % log_interval == 0:
                print("epoch {} batch id {} test loss {} test acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# 학습 모델 저장
PATH = '/content/drive/MyDrive/아이펠톤/감성대화/KcE/' 
torch.save(model, PATH + 'Koelecta-base_model_ac42.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'Koelecta-base_model_ac42_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'Koelecta-base_model_ac42_optimizer_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능


In [ ]:
#https://velog.io/@na2na8/ELECTRA%EB%A1%9C-Binary-Classification

In [ ]:
#model.save_pretrained("/content/drive/MyDrive/아이펠톤/감성대화/KcE/hi")

In [ ]:
!pip install torchinfo

In [ ]:
from torchvision import models
from torchinfo import summary

In [ ]:
model = models.vgg16()

In [ ]:
summary(model)